In [81]:
import json
import pandas as pd

In [115]:
#with open('../../../exports/dataset/results/balanced_agqa_preds/preds_psac.json', 'r') as f:
#    preds = json.load(f)


In [104]:
with open('../../../exports/dataset/results/steps_agqa_preds/preds_psac-2.json', 'r') as f:
    preds = json.load(f)


In [105]:
preds[0]

{'index': 0,
 'csv_q_id': 'YSKX3-1690',
 'id': 0,
 'question': 'After tidying up the object they were in front of, did they interact with some food?',
 'prediction': 'Yes',
 'answer': 'No',
 'prediction_success': False}

# Sort and process

In [116]:
# step 1: put into by video
by_v = {}

for p in preds:
    v_id = p['csv_q_id'][:5]
    if v_id not in by_v:
        by_v[v_id] = []
    by_v[v_id].append(p)

In [117]:
# step 2: go through each videoqs and get the num steps per question also the template
for v in by_v:
    ps = by_v[v]
    
    with open('../../../exports/dataset/balanced/test/%s.txt' % v, 'r') as f:
        QA = json.load(f)
    
    for p in ps:
        q_id = p['csv_q_id']
        q = QA[q_id]
        
        p['steps'] = q['steps']
        p['type'] = q['attributes']['type']
        p['struct'] = q['attributes']['structural']
        



In [148]:
# step 3: sort by template, answer, and num steps etc
sort = {}
sort_ans = {}
sort_struct = {}
sort_bo = {'binary': [], 'open': []}
sort_steps = {}

for v in by_v:
    ps = by_v[v]
    
    for p in ps:
        t = p['type']
        s = p['steps']
        st = p['struct']
        a = p['answer']
        
        if t not in sort:
            sort[t] = {}
            
        if s not in sort[t]:
            sort[t][s] = []
            
        sort[t][s].append(p)
        
        if a not in sort_ans:
            sort_ans[a] = []
            
        sort_ans[a].append(p)
        
        if st not in sort_struct:
            sort_struct[st] = []
            
        sort_struct[st].append(p)
        
        
        if st == 'query':
            sort_bo['open'].append(p)
        else:
            sort_bo['binary'].append(p)
            
            
        if t == 'actLast':
            
            if s not in sort_steps:
                sort_steps[s] = []
            sort_steps[s].append(p)



In [149]:
steps = []
for i in sort_steps:
    print(i, acc(sort_steps[i]), len(sort_steps[i]))
    
    x = {'steps': i, 'acc': acc(sort_steps[i]), 'number': len(sort_steps[i])}
    steps.append(x)
    
print(steps)

1 3.9 77
3 11.54 26
4 13.79 29
[{'steps': 1, 'acc': 3.9, 'number': 77}, {'steps': 3, 'acc': 11.54, 'number': 26}, {'steps': 4, 'acc': 13.79, 'number': 29}]


# Analyze

In [133]:
# function for getting accuracy of list

def acc(ps):
    cor = 0
    for p in ps:
        if p['prediction_success']:
            cor += 1
            
    return round(cor / len(ps) * 100, 2)


In [134]:
for i in sort:
    steps = list(sort[i].keys())
    ps = []
    for s in steps:
        ps += sort[i][s]
    print(acc(ps), len(ps), steps, i)

34.62 281105 [5, 4, 2, 6] objWhat
50.0 44836 [7, 6, 5] objTime
49.94 3258 [5] actLengthLongerVerify
42.6 3258 [5] actLengthShorterChoose
42.73 3258 [5] actLengthLongerChoose
49.99 21708 [7, 6, 5] actTime
49.78 22480 [6, 5, 3, 4, 7, 8] objFirstVerify
2.33 558 [4, 3, 1] actFirst
16.8 20892 [5, 2, 4, 6] objLast
49.91 20796 [6, 4, 3, 5, 7, 8] objLastVerify
50.0 21018 [5, 6] relTime
17.44 30794 [2, 5, 4, 6] objFirst
46.04 83862 [5, 3, 4, 7, 6, 8] objWhatChoose
58.89 4143 [4, 3, 6, 5, 7, 8] objLastChoose
49.93 21526 [4, 1, 3, 5] objExists
49.98 20294 [4, 5, 3, 2, 1, 6] objRelExists
49.9 21390 [4, 1, 3, 5] relExists
49.98 17066 [7, 6, 5, 4, 8, 3] xorObjRelExists
12.78 2879 [3, 4, 1, 5] objWhatGeneral
49.94 17086 [6, 5, 4, 3, 7, 8] andObjRelExists
44.82 2990 [4, 3, 5, 6, 7, 8] objFirstChoose
2.67 262 [3, 4, 1] actWhatAfterAll
4.15 289 [2] actLongest
49.53 3176 [5] actLengthShorterVerify
7.58 132 [1, 3, 4] actLast
0.0 16 [2] actShortest
11.11 135 [3, 4] actWhatBefore


In [114]:
for st in sort_struct:
    print(acc(sort_struct[st]), st, len(sort_struct[st]))

49.98 verify 33612
50.08 compare 27046
38.47 choose 16229
14.81 query 1249
48.04 logic 11510


In [112]:
for i in sort_bo:
    print(acc(sort_bo[i]), i)

47.65 binary
14.81 open


In [113]:
acc(preds)

47.19

# things to investigaet

Why is compare so high???
Do I need to take out certain q_id???

ohhhhhh
ok try taking out q_ids in everything :( 


though thats only train right??? not test?


So compare is being the issue again

yeah its really just those three...


but its ok for balanced.........
UGH
ok

ok see on answers if its the same...

OR WE do a similar question id removal process for steps. WIll need to see if that ends up changing the overall results and or meaning that we need to make new stwps issue. Im not sure though


cause i would delete by program???? that might make fewer steps. idk if it would leave enoguh though tbd


In [48]:
for i in sort_ans:
    print(acc(sort_ans[i]), i)

6.23 No
93.53 Yes
62.21 before
20.41 window
32.53 dish
81.65 after
22.45 paper
65.32 table
26.39 chair
25.22 food
47.17 mirror
63.95 doorway
39.6 closet
0.0 holding a blanket
0.0 undressing themselves
44.18 bed
51.4 doorknob
27.16 clothes
28.35 blanket
13.48 shoe
29.53 bag
34.75 door
6.8 broom
52.88 laptop
27.31 box
15.87 refrigerator
0.0 tidying something on the floor
0.0 putting a broom somewhere
10.0 standing up
0.0 sitting in a bed
44.44 floor
0.0 opening a door
1.75 picture
23.18 phone
0.0 turning off a light
24.21 pillow
11.76 television
0.0 putting a blanket somewhere
0.0 holding a phone
0.0 grasping onto a doorknob
42.67 light
0.0 closing a closet
0.0 watching something in a mirror
0.0 dressing themselves
1.43 medicine
0.0 walking through a doorway
0.0 throwing clothes somewhere
0.0 going from standing to sitting
0.0 putting some food somewhere
0.0 taking food from somewhere
0.0 fixing a doorknob
0.0 smiling at something
100.0 making some food
0.0 throwing a blanket somewhere
0

In [57]:
before = sort_ans['before']
after = sort_ans['after']

objTime = sort['objTime'][7]
relTime = sort['relTime'][6]
actTime = sort['actTime'][7]


In [58]:
len(before), len(after), len(before) - len(after)

(13495, 13551, -56)

In [59]:
len(objTime), len(relTime), len(actTime)

(13729, 9998, 3319)

In [61]:
acc(objTime), acc(relTime), acc(actTime)

(78.85, 65.09, 64.09)

# Test 1

Does the steps test have the deleted balanced? 

or does compo???


ok so i think balanced and blind should be fine. but i think compo might be messed up. 

though balanced before q_ids? so im not sure


The problem is that I dont know what happened in the deleting q_ids process. though maybe i also did it froma ll?

So the things I did is i deleted questions, and i change question indexes

maybe i look through balanced and balanced before q_id deletion? to see. also see whats different in dataset separating for compo steps



Conclusion: I do think we deleted the training steps, so it should be fine.... 

In [66]:
from os import listdir
from os.path import isfile, join

def filenames(mypath):
    onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
    return onlyfiles

In [80]:
identical = {True: 0, False: 0}

diff_QA = None
diff_QA_before = None

v_ids = filenames('../../../exports/dataset/balanced/train/')
for v in v_ids:
    
    with open('../../../exports/dataset/balanced/train/%s' % v, 'r') as f:
        QA = json.load(f)
    
    #with open('../../../exports/dataset/balanced-before-q_ids/train/%s' % v, 'r') as f:
    # with open('../../../exports/dataset_separating for compo-steps/balanced_combined/train/%s' % v, 'r') as f:
    with open('../../../exports/dataset_separating for compo-steps/balanced before trying to smooth beforeafter/train/%s' % v, 'r') as f:
        QA_before = json.load(f)
    
    
    identical[len(QA) == len(QA_before)] += 1
    
    if len(QA) != len(QA_before):
        diff_QA = QA
        diff_QA_before = QA_before
        print(v, len(QA), len(QA_before))
        

E6IL3.txt 134 12
7T6MB.txt 271 22
IJ3QB.txt 638 40


FileNotFoundError: [Errno 2] No such file or directory: '../../../exports/dataset_separating for compo-steps/balanced before trying to smooth beforeafter/train/DSWBA.txt'

In [77]:
identical

{True: 7786, False: 1}

In [72]:
len(v_ids)

7787

In [73]:
len(diff_QA), len(diff_QA_before)

(141, 238)

# Test 1.5

compare the q_ids of the balanced_csv omer had sent (tbh just download from the klone) to the balanced folder

In [100]:
csv = pd.read_csv('../../../../../Desktop/train-balanced.csv')

In [101]:
keys = list(csv['key'].values)
len(keys)

1441052

In [102]:
# step 1: put into by video
by_v = {}

for k in keys:
    v_id = k[:5]
    if v_id not in by_v:
        by_v[v_id] = []
    by_v[v_id].append(k)

In [103]:
for v in by_v:
    
    with open('../../../exports/dataset/balanced/train/%s.txt' % v, 'r') as f:
        QA = json.load(f)
        
    if len(by_v[v]) != len(QA):
        print(v, len(by_v[v]), len(QA))
        break

In [88]:
len(v_ids), len(by_v)

(7787, 7062)

# Ok 

so it does look like im right

needed to train compo/steps on something else
UGH
i am stupid

ok thats ok this is solveable



# The GODDAMN plan

1. [X] Make plan
2. [X] crochet and youtube mind break
3. [X] create a copy of the current balanced
4. [X] Get rid of the problematic q_ids in the folder. I should probaly just use the CSV for this. 
5. [] Update Omer
6. Re make the compo and steps csvs with these updated folders
7. re-train compo and steps
8. Do the inital analysis on eerything else, but with maybe getting rid of compare for now



# Test 2


see accruacy of this subset of q_ids in the normal balanced predition

that way can know if its a training problem or a subset proble

In [ ]:
#with open('../../../exports/dataset/results/balanced_agqa_preds/preds_psac.json', 'r') as f:
#    preds = json.load(f)
